# notebook n03: Freezing/Darting Analysis

Jose Oliveira da Cruz, PhD  | LeDoux Lab  
jose.cruz@nyu.edu  

<img src="LedouxLab_logo.jpg" style="width: 300.464px; height: 100px; margin: 0px;">   

This notebooks takes the .csv output from nb02

A) ``_individual_summary_stats.csv``

1. Load data from nb01 (ie `_individual_preprocessing_dlc.csv`) and nb02 (`_motion_analysis_dlc.csv`)
2. Fetch animal information
3. Create shell record with basic experiment/rat information
4. Calculate darting
5. Calculate freezing
6. Calcultate total distance
7. Calculate average speed
8. Save the data

In [ ]:
# Import dependencies
import pandas as pd
import os
import sys

# Import my code
from behavior.utils.organization import *
from behavior.analysis.motion_analysis import *
from behavior.analysis.freezing_analysis import *
from behavior.visualization.plot_events import *

## Specify where to save the data for each step

In [ ]:
step_a_save_dir = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\processed\EXP004\individual_summary_stats'

print(f'Does the directory exist? \n a: {os.path.isdir(step_a_save_dir)}') 

# Step A: ``_individual_summary_stats.csv``
The code bellow will calculate:
1. Load data from nb01 (ie `_individual_preprocessing_dlc.csv`) and nb02 (`_motion_analysis_dlc.csv`)
2. Fetch animal information
3. Create shell record with basic experiment/rat information
4. Calculate darting
5. Calculate freezing
6. Calcultate total distance
7. Calculate average speed
8. Save the data

## 1) Load necessary data files

### 1.1) Load `_individual_preprocessing_dlc.csv` from nb01b

In [ ]:
dpath_preprocessing = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\individual_preprocessing_dlc'
fpath_preprocessing = 'jc_exp004_20200110_tes01_r_286600_t00_individual_preprocessing_dlc.csv'

print(f'File exists?\n- {os.path.isfile(os.path.join(dpath_preprocessing, fpath_preprocessing))}')

In [ ]:
preprocessing_dataframe = pd.read_csv(os.path.join(dpath_preprocessing, fpath_preprocessing), index_col=0)
preprocessing_dataframe.head()

### 1.2) Load data file `_motion_analysis_dlc.csv` from nb02

In [ ]:
dpath_motion = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\motion_analysis_dlc'
fpath_motion = 'jc_exp004_20200110_tes01_r_286600_t00_motion_analysis_dlc.csv'

print(f'File exists?\n- {os.path.isfile(os.path.join(dpath_motion, fpath_motion))}')

In [ ]:
motion_dataframe = pd.read_csv(os.path.join(dpath_motion, fpath_motion), index_col=0)
motion_dataframe.head()

## 2) Fetch information about a specific rat

The code bellow will read the video key( e.g `JC_EXP005_20200124_TES01_R_287073_T00`) and search for the complete information about this specific animal.  
Then it creates an instance of the Animal class with the complete information of the animal.

### 2.1) Load Animal Record and Experiment information

In [ ]:
# Where is the main record?
main_record_directory = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\animal_record\main_record'
main_record_basename = 'main_record_20200325_151527.csv'

main_record = os.path.join(main_record_directory,
                           main_record_basename)


# Where is the information about the experiment?
experiment_info_directory = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\bonsai_extraction_led_epochs_frame_rate'
experiment_info_basename = 'JC_EXP004_20200110_TES01_cs_index_plus_frame_rate.csv'
experiment_info = os.path.join(experiment_info_directory,
                               experiment_info_basename)

### 2.2) Fetch animal information

In [ ]:
# Search object with regular expressions
pattern = r'(\w\w_\w\w\w\d\d\d_\d\d\d\d\d\d\d\d_\w\w\w\d\d_\w_\d\d\d\d\d\d_\w\d\d)'
video_key = re.search(pattern, fpath_motion).group()

# Fetch information
rat = fetch_animal_info(
    video_key, 
    main_record, 
    experiment_info,
)

## 3) Create individual_summary_stats shell (named bellow as ``df``) to begin storing the data
This step can be used to update any basic information about the animal

In [ ]:
df = create_basic_working_record(rat, 15)
df.head()

## 4) Calculate freezing per cs and per epoch and concatenate with ``df``

Freezing will be calculated as:
1. ``freezing_raw``: total freezing time per epoch
2. ``freezing_norm``: total freezing time per epoch divided by epoch duration

### 4.1) Calculate freezing 

In [ ]:
mean_freezing = calculate_mean_freezing(motion_dataframe)

### 4.2) Concatenate with ``df``

In [ ]:
df = pd.concat([df, mean_freezing], axis=1)

In [ ]:
# Freezind during cs (ie. 'peri_cs') 
df[df['cs_epoch'] == 'peri_cs']

## 5) Calculate darting per cs and per epoch and concatenate with ``df``

Darting will be calculated as:
1. ``darting_raw``: total darting time per epoch
2. ``darting_norm``: total darting time per epoch divided by epoch duration
3. ``darting_events``: number of darting events

### 5.1) Calculate Darting

In [ ]:
mean_darting = calculate_mean_darting(motion_dataframe)

### 5.2) Concatenate with ``df``

In [ ]:
df = df.merge(mean_darting, on=['cs_id', 'cs_epoch'])
df.head()

## 6) Calculate total distance

Total distance:
1. ``total_distance_cm``: measured by the ``ed_back_head`` coordinnates. Units in ``cm``.  


### 6.1) Calculate the total distance

In [ ]:
distance_per_epoch = calculate_distance_per_cs_epoch(
    preprocessing_dataframe,
    bodypart='ed_upper_torso',
    units='cm',  
)

### 6.2) Concatenate with ``df``

In [ ]:
df = df.merge(distance_per_epoch, on=['cs_id', 'cs_epoch'])
df.head()

# 7) Calculate average speed per cs/cs_epoch

1. ``mean_speed_cm/sec``:


### 7.1) Calculate the average speed

In [ ]:
units = 'cm/sec'
bodypart = 'speed_upper_torso'

speed = calculate_mean_speed_per_epoch(preprocessing_dataframe, bodypart=bodypart, units=units)

### 7.2) Concatenate with ``df``

In [ ]:
df = df.merge(speed, on=['cs_id', 'cs_epoch'])
df

## 8) Export the dataframe as .csv files

Export individual analysis. Output will be to processed data. No manual modification can be performed on this file. 

In [ ]:
saving_basename = f'{rat.video_basename}_individual_summary_stats.csv'.lower()

df.to_csv(os.path.join(step_a_save_dir, saving_basename))

# B) Merge dataframes from section A

This step will search all the previously create dataframes at a specific location, and merge them in a single dataframe ready for analysis. Remove the `#` before calling the function.

In [ ]:
#concatenate_transformed_dataframes(step_a_save_dir, save_at_directory=True)